<a href="https://colab.research.google.com/github/VayuSarangam/cmbs-cashflow/blob/main/notebooks/01_loan_level_cashflow_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone cmbs-cashflow repository

In [1]:
!git clone https://github.com/VayuSarangam/cmbs-cashflow.git
%cd cmbs-cashflow
!pip -q install -r requirements.txt
!mkdir -p data outputs src notebooks


Cloning into 'cmbs-cashflow'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 9.70 KiB | 9.70 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/cmbs-cashflow


Upload the dataset zip into Colab

In [2]:
from google.colab import files
files.upload()


Saving 01_cmbs_new_issue_projection.zip to 01_cmbs_new_issue_projection.zip


{'01_cmbs_new_issue_projection.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xec&M\\\x86\xdba\xca\x05\n\x00\x00\xbb\x1c\x00\x00\x1c\x00\x00\x00cmbs_loan_tape_new_issue.csv\x9dYKS\x1bI\x12\xbe\xef\xafP\xf8\\\xea\xa8\xacWV\x1de\xb0g\x99\x00\xe10^\xcfpr\xf4@\x83\x15+$B\x88\xd9\xe1\xdf\xcf\x97\xf50\xd2\\\xb6\xf0\x05JMG\xab\xbe\xce\xfc\x1eY\x9cN\xe3\xfa\xecT-\x9e.\xefN\xc7\xfd\xa4\xce\xb7\xe3\x06\x9f?\xed\xb6\x8f\xd3n\xffr\xb0\xfc\xf2\xf28\xa9\x8b\xab\xc5r|\x98\xd4\xd5^n\xbe\xdc\xad\xeeW\x9bq\xfd~\\\x8f\x9b\x9b\xe9\xdbb\x7f\xf2\xbc\xdf\xde\xdd\xa9\xe5v?}\xc6\x1d\xafW\xce.?\xae\xc7\xfb\xd7\xcf\x8b\x87\xedn\xffe\xda=\\l7\xfb\xefO\xea\xf3\xf40\xae6\xab\xcd\xfd\xc15<v\xbd\xddn\xea\xd3?\xdd\xec\xd5\xf2\xf2\xec\xf5\x11\x9777\xcf\x8f\xf8\xcb\x0b\xfer\xf0\xe0\xc7\xc7\xdd\xb8z\x9an\xbf\x8e\xeb\xe7\x83\r\x9c\x7f\xf9\xfa\xfa\xe1\xf4\xea\xe4\xf3\xeb\xa7\xabi\xf7\xe7\xea\x06\xdf\xfdq\x9a\xde?\xca^\xeeVW\x8f\xbbi\xbc\xbd\xfcs\xda\x9dmn\xa7\xbf\xbe\xfd\x81\xeb\xa7\xd3\x1f\xfb\xeb\xd5\xb4\xbe\x95\xad|]m\xf6\x

Unzip into data/:

In [3]:
!unzip -o 01_cmbs_new_issue_projection.zip -d data
!ls -lah data


Archive:  01_cmbs_new_issue_projection.zip
  inflating: data/cmbs_loan_tape_new_issue.csv  
  inflating: data/cmbs_deal_terms.csv  
  inflating: data/scenario_shocks.csv  
  inflating: data/OUTPUT_loan_level_projection_TEMPLATE.csv  
  inflating: data/OUTPUT_collateral_cashflows_TEMPLATE.csv  
  inflating: data/README.txt         
total 40K
drwxr-xr-x 2 root root 4.0K Feb 15 03:20 .
drwxr-xr-x 7 root root 4.0K Feb 15 03:19 ..
-rw-r--r-- 1 root root  230 Feb 13 04:55 cmbs_deal_terms.csv
-rw-r--r-- 1 root root 7.2K Feb 13 04:55 cmbs_loan_tape_new_issue.csv
-rw-r--r-- 1 root root    1 Feb 15 03:17 .gitkeep
-rw-r--r-- 1 root root  141 Feb 13 04:55 OUTPUT_collateral_cashflows_TEMPLATE.csv
-rw-r--r-- 1 root root  198 Feb 13 04:55 OUTPUT_loan_level_projection_TEMPLATE.csv
-rw-r--r-- 1 root root  783 Feb 13 04:55 README.txt
-rw-r--r-- 1 root root  334 Feb 13 04:55 scenario_shocks.csv


Check if you are in repo folder to save the work in GitHub

In [4]:
%cd /content/cmbs-cashflow
!git status


/content/cmbs-cashflow
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	01_cmbs_new_issue_projection.zip
	data/OUTPUT_collateral_cashflows_TEMPLATE.csv
	data/OUTPUT_loan_level_projection_TEMPLATE.csv
	data/README.txt
	data/cmbs_deal_terms.csv
	data/cmbs_loan_tape_new_issue.csv
	data/scenario_shocks.csv

nothing added to commit but untracked files present (use "git add" to track)
